# Example to add thermal noise to simulations
This notebook provides an example of how to add thermal noise as different components.\
The identified components are:
* Ice
* Galaxy
* Electronics
  
Each of these components uses a different module from NuRadio

In [54]:
import datetime
import logging
import matplotlib.pyplot as plt
import numpy as np
import os

import NuRadioReco
from NuRadioReco.detector.RNO_G.rnog_detector import Detector
from NuRadioReco.framework.event import Event
from NuRadioReco.framework.station import Station
from NuRadioReco.framework.channel import Channel
from NuRadioReco.utilities import units

We start by generating a set of events with the same structure as simulated events. We do this to use the module.run() convention from NuRadio

In [46]:
sampling_rate = 3.2 * units.GHz
nr_samples = 2048
frequencies = np.fft.rfftfreq(nr_samples, d=1./sampling_rate)

def generate_event():
    event = Event(run_number=-1, event_id=-1)
    station = Station(0)
    station.set_station_time("2023-08-01", format="isot")
    channel_ids = [0]
    for channel_id in channel_ids:
        channel = Channel(channel_id)
        channel.set_frequency_spectrum(np.zeros_like(frequencies, dtype=np.complex128), sampling_rate)
        station.add_channel(channel)
    event.set_station(station)
    return event

nr_events = 20
events = [generate_event() for e in range(nr_events)]

## Ice noise

In [47]:
eff_temp_dir = os.path.join(os.path.dirname(NuRadioReco.__file__), "examples", "RNOG", "eff_temperatures")

In [ ]:
from NuRadioReco.modules.channelThermalNoiseAdder import channelThermalNoiseAdder

detector = Detector(database_connection="RNOG_public", log_level=logging.NOTSET,
                   select_stations=11)
detector_time = datetime.datetime(2023, 8, 1)
detector.update(detector_time)

ice_noise = channelThermalNoiseAdder()
ice_noise.begin(sim_library_dir=eff_temp_dir)
for event in events:
    station = event.get_station()
    ice_noise.run(event, station, detector=0)

## Electronic noise

In principle the electronic noise should be frequency dependent and is ideally derived from a noise temperature measurement. Since these are not available the electronic noise is approximated as flat with a certain noise temperature. From old measurements (see noise measurements in https://arxiv.org/abs/2411.12922 Figure 8) we approximate this temperature as 80 K.

In the context of the RNO-G system absolute amplitude calibration, an extra free parameter is added to the electronic noise profile to account for this uncertainty. This free parameter module is not included in this example.

## Galactic noise